<a href="https://colab.research.google.com/github/redrum88/autoevolution/blob/main/step_2_all_models_from_brands.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd

INPUT_FILE = 'brand_names.csv'
OUTPUT_FILE = "all_model_variants.csv"

brands_df = pd.read_csv(INPUT_FILE)

# Convert Brand URL column values to list
url_list = brands_df['Brand URL'].tolist()
brand_name_list = brands_df['Brand Name'].tolist()
brand_logo_url_list = brands_df['Brand Logo'].tolist()
len(url_list), len(brand_name_list), len(brand_logo_url_list)



In [ ]:

PATH_DIR = "data/models/"

# Create folder if not exist
if not os.path.exists(PATH_DIR):
    os.makedirs(PATH_DIR)

for idx, row in brands_df.iterrows():
    url = row['Brand URL']
    name = row['Brand Name']
    logo_url = row['Brand Logo']

    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    cars = soup.find_all('div', class_='carmod')
    car_data = []

    for car in cars:
        car_model = car.find('h4').text.strip()
        car_years = car.find('div', class_='col3width').find_all('span')[0].text.strip()
        car_link = car.find('a')['href']
        car_image = car.find('img')['src']
        car_dict = {'Model': car_model,
                    'Years': car_years,
                    'Link': car_link,
                    'Image_Link': car_image,
                    'Brand URL': url,
                    'Brand': name,
                    'Brand Logo URL': logo_url}
        car_data.append(car_dict)

    #print(car_dict)
    df = pd.DataFrame(car_data)
    save_path = PATH_DIR
    file_name = url.split("/")[-2]
    df.to_csv(os.path.join(save_path, file_name + '.csv'), index=False)

    print(f"{len(car_data)} car models extracted from {name}")


In [ ]:
import os
import glob
import pandas as pd


# Input directory
input_dir = "data/models/"

# Output directory
output_dir = "data/"

# Find all .csv files in input_dir
csv_files = glob.glob(os.path.join(input_dir, "*.csv"))

# Initialize dataframe
combined_data = pd.DataFrame()

# Loop each .csv file
for csv_file in csv_files:
    data = pd.read_csv(csv_file)
    combined_data = pd.concat([combined_data, data])

# Save combined data to .csv file
combined_data.to_csv(os.path.join(output_dir, OUTPUT_FILE), index=False)
